# Préparation des données : Conversion des XML en CSV

Cette feuille lit l'ensemble des XML représentant les données et les sauvegarde avec le format CSV.

In [1]:
INPUT_PATH_TO_XML_FILES="data"
OUTPUT_PATH_TO_CSV_FILES="."

## Les  imports

In [2]:
import zipfile
import xml.sax
import csv
import glob
import logging
import shutil
import os

In [3]:
logging.basicConfig(level='INFO')

In [4]:
os.chdir(OUTPUT_PATH_TO_CSV_FILES)
print('current work dir is:', os.getcwd())

current work dir is: /home/houcine/workspace/school-eda-serviceStations


## Création des CSV

In [5]:
# Création du CSV qui contient les données des points de ventes
pdvcsvfile = open('pdv.csv', 'w')
pdvnames   = ["annee",
              "id",
              "latitude",
              "longitude",
              "cp",
              "pop",
              "ouverture_debut",
              "ouverture_fin",
              "ouverture_saufjour",
              "adresse",
              "ville"]
pdv_writer = csv.DictWriter(pdvcsvfile, fieldnames=pdvnames)
pdv_writer.writeheader()

In [6]:
# Création du CSV qui contient les données des ruptures
rupturescsvfile = open('ruptures.csv', 'w')
rupturesnames = ["annee",
                 "id_pdv",
                 "id",
                 "nom",
                 "debut",
                 "fin"]
ruptures_writer = csv.DictWriter(rupturescsvfile, fieldnames=rupturesnames)
ruptures_writer.writeheader()

In [7]:
# Création du CSV qui contient les données des fermetures
fermeturescsvfile = open('fermetures.csv', 'w')
fermeturesnames = ["annee",
                   "id_pdv",
                   "type",
                   "debut",
                   "fin"]
fermetures_writer = csv.DictWriter(fermeturescsvfile, fieldnames=fermeturesnames)
fermetures_writer.writeheader()

In [8]:
# Création du CSV qui contient les données des prix
prixcsvfile = open('prix.csv', 'w')
prixnames   = ["annee",
              "id_pdv",
              "nom",
              "id",
              "maj",
              "valeur"]
prix_writer = csv.DictWriter(prixcsvfile, fieldnames=prixnames)
prix_writer.writeheader()

In [9]:
# Création du CSV qui contient les données des services
servicescsvfile = open('services.csv', 'w')
servicesnames   = ["annee",
                   "id_pdv",
                   "service"]
services_writer = csv.DictWriter(servicescsvfile, fieldnames=servicesnames)
services_writer.writeheader()

In [10]:
# Initialisation des dictionnaire des lignes temporiaires à inserer dans les csv
pdv_row        = dict.fromkeys(pdvnames)
ruptures_row   = dict.fromkeys(rupturesnames)
fermetures_row = dict.fromkeys(fermeturesnames)
prix_row       = dict.fromkeys(prixnames)
services_row   = dict.fromkeys(servicesnames)

## le SaxParserHandler

Il existe plusieurs méthodes pour lire un xml. Ici on a choisi **xml.sax**

In [11]:
# verfier tous les tags non traités
tags_used = ["pdv", "ville", "ouverture", "fermeture", "rupture", "prix", "service", "services", "adresse"]
unused_tags = []

In [12]:
class PdvHandler(xml.sax.ContentHandler):
    def __init__(self, annee): # on passe l'année en parametre au constructeur
        self.CurrentData = ""
        self.annee       = annee
        self.id_pdv      = ""
        self.adresse     = ""
        self.ville       = ""
        self.service     = ""
        self.isVille     = False
      
   # Call when an element starts
    def startElement(self, tag, attributes):
        
        # catch all unused tags
        if not tag in tags_used:
            unused_tags.append(tag)

            
        self.CurrentData = tag
        # quand on trouve la balise pdv
        if tag == "pdv":
            pdv_row['annee']= self.annee       # on commence l'implementation de la ligne à inserer
                                            # dans le pdvCsv
            self.id_pdv = attributes['id']     # on garde le id du pdv pour le rajouter dans les autres 
                                            # CSV comme clé primaire
            if attributes.getNames() != []:
                #id, latitude, longitude, cp, pop
                for name in attributes.getNames():
                    pdv_row[name] = attributes.getValue(name)

        # quand on trouve la balise ouverture
        elif tag == "ville":
            self.isVille = True
        elif tag == "ouverture":
            if attributes.getNames() != []:
                #debut, fin, saufjour
                for name in attributes.getNames():
                    pdv_row["ouverture_"+name] = attributes.getValue(name)
            
        # quand on trouve la balise fermeture
        elif tag == "fermeture":
            fermetures_row["annee"] = self.annee
            fermetures_row["id_pdv"] = self.id_pdv
            if attributes.getNames() != []:
                #type, debut, fin
                for name in attributes.getNames():
                    fermetures_row[name] = attributes.getValue(name)
                
        # quand on trouve la balise rupture
        elif tag == "rupture":
            ruptures_row["annee"] = self.annee
            ruptures_row["id_pdv"] = self.id_pdv
            if attributes.getNames() != []:
                #id, nom, debut, fin
                for name in attributes.getNames():
                    ruptures_row[name] = attributes.getValue(name)
                    
        # quand on trouve la balise prix
        elif tag == "prix":
            if attributes.getNames() != []:
                prix_row['annee']  = self.annee
                prix_row['id_pdv'] = self.id_pdv
                #nom, id, maj, valeur
                for name in attributes.getNames():
                    prix_row[name] = attributes.getValue(name)
        elif self.isVille:
            self.ville += tag
                
    # Call when an elements ends
    def endElement(self, tag):
        if self.CurrentData == "adresse":
            pdv_row["adresse"] = self.adresse
        elif self.CurrentData == "ville":
            self.isVille = False
            pdv_row["ville"] = self.ville
            self.ville = ""
        elif self.CurrentData == "service":
            services_row['annee']   = self.annee
            services_row['id_pdv']  = self.id_pdv
            services_row["service"] = self.service
            services_writer.writerow(services_row) # insertion de la ligne dans le csv
        elif self.CurrentData == "prix":
            prix_writer.writerow(prix_row)         # insertion de la ligne dans le csv
        elif tag == "pdv":
            pdv_writer.writerow(pdv_row)           # insertion de la ligne dans le csv
            self.CurrentData = ""
        elif tag == "rupture":
            ruptures_writer.writerow(ruptures_row)           # insertion de la ligne dans le csv
            self.CurrentData = ""
        elif tag == "fermeture":
            fermetures_writer.writerow(fermetures_row)           # insertion de la ligne dans le csv
            self.CurrentData = ""
    # Call when a character is read
    def characters(self, content):
        if self.CurrentData == "adresse":
            self.adresse = content
        elif self.CurrentData == "ville" or self.isVille:
            self.ville += content.strip()
        elif self.CurrentData == "service":
            self.service = content
  

## Traitement des Zip et Parsing des XML dans les CSV

In [13]:
def execute():
    # Pour chaque fichier zip dans 'INPUT_PATH_TO_XML_FILES' et qui commence par 'PrixC' et se termine par '.zip'
    for pathZip in glob.glob(INPUT_PATH_TO_XML_FILES+'/PrixC*.zip'):
        annee   = pathZip[-8:-4]
        nameXml = pathZip[-30:-3]+"xml"

        # lire le zip
        logging.debug("Opening ZIP file " + pathZip)    
        zf = zipfile.ZipFile(pathZip, 'r')
        # lire le XML
        f  = zf.open(nameXml)
        # créer le XMLReader
        parser = xml.sax.make_parser()
        # Construire le Handler
        Handler = PdvHandler(annee)
        # Override le superHandler
        parser.setContentHandler(Handler)
        # Parser
        logging.debug("Parsing XML file " + nameXml)
        parser.parse(f)

        # fermer le zip
        zf.close()

## Execution du parsing

je l'execute en utilisant ***%time*** pour comparer avec ***lxml***

In [14]:
%time execute()

CPU times: user 5min 34s, sys: 2.04 s, total: 5min 36s
Wall time: 5min 39s


les resultats du **lxml** : <br>
CPU times: user 5min 32s, sys: 3.04 s, total: 5min 35s<br>
Wall time: 5min 37s

On trouve la même performance pour les deux méthodes. <br>
et vu la lisibilité apporté par **lxml**, il faudra, donc, la préviligié plus que **xml.sax**

In [15]:
set(unused_tags)

{'horaire', 'horaires', 'jour', 'pdv_liste'}

on trouve que sur les autres versions des données XML, les balises  ***'horaire', 'horaires', 'jour'*** apparaissent. <br>
il sera bien de les ajouter et de voir ce qu'il peuvent nous apporter comme informations. <br>
mais vu que j'avais commencé beacoup de mon travail sans faire attention a ce detail, j'ai décidé de ne pas mettre à jour les CSV.

## Fermeture des CSV ouverts

In [16]:
pdvcsvfile.close()
rupturescsvfile.close()
fermeturescsvfile.close()
servicescsvfile.close()
prixcsvfile.close()

## Verification du nombre de ligne insérées

In [17]:
print("{:10} {}\n".format("CSVs","nb de lignes"))
for csv in glob.glob('*.csv'):
    with open(csv) as c:
        print("{:10} {}".format(csv[:-4],len(c.readlines())))

CSVs       nb de lignes

prix       28556968
fermetures 139614
pdv        136716
ruptures   260876
services   962905
